In [72]:
import numpy as np
import cudf
import cuspatial
from shapely.geometry import Point, Polygon
import geopandas
import time 



N = 10
lenpoly = 1000
N_polys = 40
polygon_list = [None]*N_polys
for j in range(N_polys):
    shift = np.random.uniform(-1,1)
    polygon = [[np.sin(x)+shift,np.cos(x)+shift] for x in np.linspace(0,2*np.pi,lenpoly)]
    polygon_arr = np.array(polygon)
    polygon_list[j] = polygon_arr

points = np.random.uniform(-1.5, 1.5, size=(N, 2))

st = time.time()
points_list = [Point(points[i,:]) for i in range(points.shape[0])]
shapekly_result_list = [None]*N_polys
for j in range(N_polys):
    shapely_poly = Polygon(polygon_list[j])
    shapely_result = shapely_poly.contains(points_list)
    shapekly_result_list[j] = shapely_result
end = time.time()

print(end-st)


st = time.time()

shapely_points_list = [Point(points[i,:]) for i in range(points.shape[0])]
shapely_poly_list = [Polygon(polygon) for polygon in polygon_list]

cupoints = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_points_list))
cupoly = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_poly_list))
current_index = 0
next_index = 0
while next_index < N_polys:
    if N_polys - current_index > 30:
        next_index = current_index + 30
    else:
        next_index = N_polys
    result = cuspatial.point_in_polygon(cupoints,
                                    cupoly[current_index:next_index]                   
                                    )
    current_index = next_index
end = time.time()

print(end-st)

st = time.time()
interleaved_points = points.flatten()
cupoints2 = cuspatial.GeoSeries.from_points_xy(interleaved_points)

shapely_poly = [Polygon(polygon) for polygon in polygon_list]
cupoly = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_poly))
print("length:",len(cupoly))


current_index = 0
next_index = 0
frames = []
while next_index < N_polys:
    if N_polys - current_index > 30:
        next_index = current_index + 30
    else:
        next_index = N_polys
    result = cuspatial.point_in_polygon(cupoints2,
                                    cupoly[current_index:next_index]                   
                                    )
    #df.update(result)
    current_index = next_index
    frames.append(result)
    
end = time.time()
print(end-st)

print(result)
result.loc[0]
result.values

df = cudf.concat(frames,axis=1)
print(df)
print(df.values[2,39])
type(df.values)

0.006578683853149414
0.15498900413513184
length: 40


/home/matthew/.local/share/virtualenvs/biopsylocalization-python-_12o-Huq/lib/python3.9/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/matthew/.local/share/virtualenvs/biopsylocalization-python-_12o-Huq/lib/python3.9/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/matthew/.local/share/virtualenvs/biopsylocalization-python-_12o-Huq/lib/python3.9/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/matthew/.local/share/virtualenvs/biopsylocalization-python-_12o-Huq/lib/python3.9/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 1 will likely res

0.1421678066253662
      30     31     32     33     34     35     36     37     38     39
0  False  False  False  False  False  False  False  False  False  False
1   True  False   True  False  False   True  False  False  False   True
2  False  False  False  False  False  False  False  False  False  False
3  False  False  False  False  False  False  False  False  False  False
4  False  False  False  False  False  False  False   True   True  False
5  False  False  False  False  False  False  False   True   True  False
6  False  False  False  False  False  False  False  False  False  False
7  False   True  False  False   True  False  False   True   True  False
8  False  False  False  False  False  False  False  False  False  False
9  False  False  False  False  False  False  False  False  False  False
      0      1      2      3      4      5      6      7      8      9   ...  \
0  False  False  False  False  False  False  False  False  False  False  ...   
1  False  False  False  False

cupy.ndarray

In [88]:
import numpy as np
import cuspatial
from shapely.geometry import Point, Polygon

N=10

points = np.random.uniform(-1.5, 1.5, size=(N, 2))
shapely_points_list = [Point(points[i,:]) for i in range(points.shape[0])]
cupoints = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_points_list))
print(cupoints)
print(cupoints[0:10])

cupoints.points


0     POINT (-1.39248 0.60009)
1     POINT (1.06403 -1.13638)
2    POINT (-0.54963 -0.82837)
3    POINT (-1.47684 -0.88845)
4     POINT (0.19736 -0.91442)
5     POINT (-1.39196 1.09605)
6     POINT (-0.87897 1.31730)
7      POINT (0.22517 0.06904)
8      POINT (1.44191 1.12307)
9     POINT (-0.89762 0.14314)
dtype: geometry
0     POINT (-1.39248 0.60009)
1     POINT (1.06403 -1.13638)
2    POINT (-0.54963 -0.82837)
3    POINT (-1.47684 -0.88845)
4     POINT (0.19736 -0.91442)
5     POINT (-1.39196 1.09605)
6     POINT (-0.87897 1.31730)
7      POINT (0.22517 0.06904)
8      POINT (1.44191 1.12307)
9     POINT (-0.89762 0.14314)
dtype: geometry


In [26]:
import numpy as np

a = np.array([[[1,1,1],[2,2,2],[3,3,3],[4,4,4]],[[1,1,1],[2,2,2],[3,3,3],[4,4,4]],[[5,1,1],[2,2,2],[3,3,3],[4,6,4]]])
a.shape[1]

a[0:2]

b = np.reshape(a,(-1,3))
print(b)
print(b[:,2])

[[1 1 1]
 [2 2 2]
 [3 3 3]
 [4 4 4]
 [1 1 1]
 [2 2 2]
 [3 3 3]
 [4 4 4]
 [5 1 1]
 [2 2 2]
 [3 3 3]
 [4 6 4]]
[1 2 3 4 1 2 3 4 1 2 3 4]


In [41]:
from bisect import bisect_left


def take_closest(myList_org, myNumber):
    """
    Assumes myList is sorted. Returns index of closest value and the closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    myList = myList_org.copy()
    if myList[0] > myList[1]:
        myList.reverse()
        list_reversed = True
    else:
        list_reversed = False
    if list_reversed == False:
        pos = bisect_left(myList, myNumber)
        if pos == 0:
            return 0, myList[0]
        if pos == len(myList):
            return pos-1, myList[-1]
        before = myList[pos - 1]
        after = myList[pos]
        if after - myNumber < myNumber - before:
            return pos, after
        else:
            return pos - 1, before
    elif list_reversed == True:
        pos = bisect_left(myList, myNumber)
        if pos == 0:
            return len(myList)-1, myList[0]
        if pos == len(myList):
            return 0, myList[-1]
        before = myList[pos - 1]
        after = myList[pos]
        if after - myNumber < myNumber - before:
            return len(myList) - 1 - pos, after
        else:
            return len(myList) - pos, before


def take_closest_array_input(myList, myArray):
    """
    Assumes myList is sorted. Returns index of closest value and the closest value to myNumber.

    If two numbers are equally close, return the smallest number. Takes an array as input and vectorizes the 'take_closest_index_only' function. 

    Returns an array of the indices of the original list that contains the value that is closest to each element of the given array.
    """

    take_closest_index_only_vectorized = np.vectorize(take_closest, excluded=['myList_org'])
    closest_vals_indices_array, closest_vals_array = take_closest_index_only_vectorized(myList_org = myList,myNumber = myArray)
    return closest_vals_indices_array, closest_vals_array


a = [1,3,5,7,8,9,10,12,14,15,16,17,18,19]

b = np.array([3.4,5.6,4.8,8.8,1.5,6.3,6.4,10.7,19,13])

c, d = take_closest_array_input(a,b)

print(c)
print(d)



[ 1  2  2  5  0  3  3  6 13  7]
[ 3  5  5  9  1  7  7 10 19 12]


In [86]:
import cupy as cp

a = [cp.array(True) for j in range(10)]

b = cp.array(a)

df = cudf.DataFrame.from_dict({"a":a})
b

color = cp.array([1,0,0],dtype=float)

color

array([1., 0., 0.])

In [83]:
a = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

for j,i in enumerate(a):
    print(j % 10)


0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


In [94]:
import cudf
import numpy as np

N=10

points = [np.array([1,1,1]),np.array([1,2,1]),np.array([1,1,2])]

dictionary = {"points": points}

a = cudf.DataFrame.from_dict(dictionary)
a

,points
0,"[1, 1, 1]"
1,"[1, 2, 1]"
2,"[1, 1, 2]"


In [95]:
[int(i) for i in range(5) for j in range(3)]

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]

In [4]:
import numpy as np

a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b=np.array([0,1])
c=np.array([1,2])
c = a[b,c]
c

array([2, 6])

In [6]:
import numpy as np

def color_by_bool_by_arr(contain_bool_arr):
    color_by_bool_vectorized = np.vectorize(color_by_bool)
    color_arr = color_by_bool_vectorized(contain_bool_arr)
    return color_arr



def color_by_bool(bool_val):
    if bool_val == True:
        color = np.array([0,1,0],dtype=float)
    elif bool_val == False:
        color = np.array([1,0,0],dtype=float)

    return color

a = np.array([True, False, True])

for element in a:
    print(element)

True
False
True


In [12]:
import cupy as cp 
import time  
import numpy as np


def color_by_bool_by_arr(contain_bool_arr):
    color_list = [None]*len(contain_bool_arr)
    for index,element_bool in enumerate(contain_bool_arr):
        if element_bool == True:
            color = np.array([0,1,0])
        else:
            color = np.array([1,0,0])
        color_list[index] = color
    return color_list

a = cp.random.randint(2, size=10000)
st = time.time()
b= color_by_bool_by_arr(a)

end = time.time()
print(end-st)
a = np.random.randint(2, size=10000)


st = time.time()
b= color_by_bool_by_arr(a)

end = time.time()
print(end-st)


1.077784538269043
0.013470649719238281


In [11]:
import cudf
import pandas
import numpy as np

example_dict = {"1":[True, False, True],"2":[False, False, True]}

df = cudf.DataFrame(example_dict)

print(df)

df.to_numpy()[[0,1,2],[1,1,1]]







       1      2
0   True  False
1  False  False
2   True   True


array([False, False,  True])

In [50]:
import numpy as np

a = np.empty([10,3])
a[2,:] = np.array([1,1,1])
a

array([[4.66881057e-310, 0.00000000e+000, 4.78104785e+180],
       [3.94811624e+180, 7.29010818e+045, 2.55388173e+151],
       [1.00000000e+000, 1.00000000e+000, 1.00000000e+000],
       [6.53042243e+015, 2.00291244e-076, 2.01843055e-076],
       [2.10479226e-076, 2.19115395e-076, 1.18027089e+180],
       [8.90643897e+179, 1.70544946e+214, 1.36936617e+069],
       [8.79819426e+252, 7.01219261e+194, 2.28221981e+243],
       [5.45385516e+069, 9.75023642e+199, 1.74552498e-076],
       [5.14283944e+025, 5.12318582e+025, 4.00234504e+035],
       [5.23279660e+261, 3.02253513e-066, 5.17130405e-310]])

In [27]:
import cupy as cp 
import numpy as np

points = np.array([[[1,1,1],[1,1,1],[1,1,1]],[[2,2,2],[2,2,2],[2,2,2]]])

points_cp = cp.array(points)

vec_arr = np.array([[0.5,0.5,0.5],[1,1,1]])

vec_arr_cp = cp.array(vec_arr)

vec_arr_cp_reshaped = cp.reshape(vec_arr_cp,(2,1,3))

vec_arr_cp_reshaped

vec_arr_cp_extended = cp.tile(vec_arr_cp_reshaped,(1,3,1))

vec_arr_cp_extended

points_cp_translated = points_cp + vec_arr_cp_extended

points_cp_translated

array([[[1.5, 1.5, 1.5],
        [1.5, 1.5, 1.5],
        [1.5, 1.5, 1.5]],

       [[3. , 3. , 3. ],
        [3. , 3. , 3. ],
        [3. , 3. , 3. ]]])

In [18]:
test = [[1,2,3],2,3]
test1 = test.copy()

def test_func(list):
    list.append(3)
    return 3

test_func(test1)

test1[0].append(1)
test


[[1, 2, 3, 1], 2, 3]

In [37]:
import numpy as np

import cupy as cp

a = np.array([1,2,3])

b = np.array([1,2,3])

c = cp.array([1,2,3])

d = cp.array(a)

e = cp.array(c)


d[0] =10 

a




array([1, 2, 3])

In [50]:
import cupy as cp

a = cp.array([[1,1,1],[2,2,2],[3,3,3]])

b = cp.resize(a,(1,3,3))
c = cp.repeat(b,3,0)

d = cp.array([1,2,3,4])
print(d)
print(d.shape)
e=d[...,None]
print(e)
e.shape

[1 2 3 4]
(4,)
[[1]
 [2]
 [3]
 [4]]


(4, 1)

In [49]:
import numpy as np

a = np.array([1,2,3,4])
print(a.shape)
b=a[...,None]
print(b.shape)
b


(4,)
(4, 1)


array([[1],
       [2],
       [3],
       [4]])

In [55]:
import cupy as cp
import numpy as np

a = cp.array([[[1,1,1],[1,1,1],[1,1,1]],[[2,2,2],[2,2,2],[2,2,2]]])

for slice in a:
    print(slice)
    print('___')

[[1 1 1]
 [1 1 1]
 [1 1 1]]
___
[[2 2 2]
 [2 2 2]
 [2 2 2]]
___


In [58]:
print("org "+str(2-1))

org 1


In [60]:
import numpy as np

a = np.array(None)
a

array(None, dtype=object)

In [55]:
import cudf 

df = cudf.DataFrame({'point idx': [0, 1, 2, 3, 0, 1, 2, 3],
                   'pt contained': [False, True, True, True, False, True, False, True]})
print(df)



a = df.groupby(['point idx']).sum().to_numpy().T
a


   point idx  pt contained
0          0         False
1          1          True
2          2          True
3          3          True
4          0         False
5          1          True
6          2         False
7          3          True


array([[0, 2, 2, 1]])

In [61]:
import numpy as np

a = np.array([[1,1,1],[2,2,2]])
b = a.flatten(order = 'C').tolist()
b


[1, 1, 1, 2, 2, 2]

In [1]:
import cupy as cp   

a = cp.array([[[1,1,1],[1,1,1]],[[2,2,2],[2,2,2]]])
b= cp.reshape(a,(-1,3))
b

array([[1, 1, 1],
       [1, 1, 1],
       [2, 2, 2],
       [2, 2, 2]])

In [26]:
import cupy as cp

a = cp.array([[1,4,3],[1,4,7],[3,10,15]])
b = (a[:,2] < 10) 
c = (a[:,2] > 5)
print(b)
print(c)
d = cp.logical_and(b,c)
d
e = [1,2,3,4,45,56]
max(e)


[ True  True False]
[False  True  True]


56

In [2]:
import numpy as np

a = np.array([[1,2,3],[4,5,6],[7,8,9]])
a[0:2,0:2] = 10
a

array([[10, 10,  3],
       [10, 10,  6],
       [ 7,  8,  9]])

In [10]:
import pandas as pd


df = pd.DataFrame({
     'col_to_group': ['A', 'A', 'B', 'B', 'A', 'B'],
     'col_1': ['a', 'b', 'c', 'a', 'a', 'a'],
     'col_2': ['x', 'y', 'y', 'y','x','x'],
     'col_3': [.1, .2, .1, .9, .7,0.5],
     'col_4': ["Nominal", 1,2, "Nominal", 1, 2],
})

df3 = df[df['col_2']!= 'x'].reset_index()

print(df3)

df2 = df.drop([i for i in range(2)])

grouped = df.groupby(['col_4'])
df1 = grouped.get_group("Nominal")
df1

   index col_to_group col_1 col_2  col_3    col_4
0      1            A     b     y    0.2        1
1      2            B     c     y    0.1        2
2      3            B     a     y    0.9  Nominal


,col_to_group,col_1,col_2,col_3,col_4
0,A,a,x,0.1,Nominal
3,B,a,y,0.9,Nominal


In [22]:
import numpy as np

a = np.array([[[1,1,1],[1,1,1],[1,1,1]],[[2,2,2],[2,2,2],[2,2,2]]])
print(a)
print(a.shape[0])
b = np.array([[0,0,0],[0,0,0],[0,0,0]])
b = np.expand_dims(b,axis=0)
print(b)
c = np.concatenate((b,a))
print(c)



[[[1 1 1]
  [1 1 1]
  [1 1 1]]

 [[2 2 2]
  [2 2 2]
  [2 2 2]]]
2
[[[0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]]

 [[1 1 1]
  [1 1 1]
  [1 1 1]]

 [[2 2 2]
  [2 2 2]
  [2 2 2]]]


In [36]:
import numpy as np

a = np.array([[1,1,1],[2,2,2],[3,3,3]])
b = np.array([4,4,4])
c = np.expand_dims(b,axis =1)
d = np.append(c,a, axis = 1)
print(d)
d.tolist()


[[4 1 1 1]
 [4 2 2 2]
 [4 3 3 3]]


[[4, 1, 1, 1], [4, 2, 2, 2], [4, 3, 3, 3]]

In [1]:
list(range(0,5))

[0, 1, 2, 3, 4]

In [6]:
import numpy as np

a=np.arange(0,10).tolist()
a.prepend('1')

AttributeError: 'list' object has no attribute 'prepend'

In [29]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
                go.Scatter(
                    name='Nominal',
                    x=[0,1,2,3,4],
                    y=[1,1,1,2,2],
                    marker=dict(color='red',
                                ),
                    #marker_line_width = 1,
                    
                    mode='markers',
                    showlegend=True
                )
            )

fig.show()

In [30]:
import numpy as np


a = np.array([1,2,3]).tolist()
a

[1, 2, 3]

In [10]:
import numpy as np

a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b = np.array([11,12,13])
c = np.expand_dims(b,axis =1 )
d = np.append(a,c,axis = 1)
print(d)

e = np.empty((a.shape[0],a.shape[1]+1))
e[:,0:3] = a
e[:,3] = b
print(e)

e[:,3]


[[ 1  2  3 11]
 [ 4  5  6 12]
 [ 7  8  9 13]]
[[ 1.  2.  3. 11.]
 [ 4.  5.  6. 12.]
 [ 7.  8.  9. 13.]]


array([11., 12., 13.])

In [7]:
list(range(0,3))

[0, 1, 2]

In [16]:
import numpy as np

a = np.empty((3,2,4))
a.fill(1)
print(a)
a[0,:,:]

[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]]]


array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [17]:
a = {}

a["1"] = 1
a
len(a)

1

In [30]:
import numpy as np

a = np.array([1,1,1])
b=np.array([2,2,2])
c=np.array([1,2,3,4])
d=np.array([[1,2,3],[1,2,3]])
print(a.shape == a.shape)
print(a.shape == b.shape)
print(a.shape == c.shape)
print(a.shape == d.shape)
len(d)
print((a - 2))
print((a - 2)**2)
print((b - 2)**2)
np.sum((a - 2)**2 + (b - 2)**2)


True
True
False
False
[-1 -1 -1]
[1 1 1]
[0 0 0]


3

In [32]:
a = {}
a['a'].append(1)

KeyError: 'a'

In [37]:
a = np.empty(10)
a.shape[0]

10

In [39]:
import numpy as np

a = np.array([1,2,3])
(a-1)**2

array([0, 1, 4])

In [40]:
1/2*3

1.5

In [66]:
a = [1,2,3]
b = [4,5,6]
c = np.empty((2,1,3))
c[0] = a
c[1] = b
c
d = np.array(a).reshape((1,-1))
d[:,0:2]

array([[1, 2]])

In [69]:
test_dict_1 = {'a':[1,2,3], 'b':[4,5,6]}

for index,(key,item) in enumerate(test_dict_1.items()):
    print(index)
    print(key)
    print(item)
    print('---')

len(test_dict_1)

0
a
[1, 2, 3]
---
1
b
[4, 5, 6]
---


2

In [79]:
import pandas as pd
import numpy as np
import cudf

technologies = {
    'ROI':["P","P","D1","D1","D2","D2","P","P","D1","D1","D2","D2","P","P","D1","D1","D2","D2",],
    'Trial':[1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3],
    'original_pt' :[1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2],
    'Contained':[True,True,True,True,True,True,True,True,False,False,False,False,False,True,True,True,False,True],
    'test': np.array([True,True,True,True,True,True,True,True,False,False,False,False,False,True,True,True,False,True])
    }
df = pd.DataFrame(technologies)
print(df)
comb = ['P',"D1"]
"""
df[(df["ROI"].isin(comb))].reset_index()[
        ["Contained","original_pt","Trial"]
        ].groupby(["original_pt","Trial"]).all().sort_index().groupby(["original_pt"]).sum().sort_index().to_numpy().T.flatten(order = 'C').tolist()
"""
df['ROI'].unique().tolist()

   ROI  Trial  original_pt  Contained   test
0    P      1            1       True   True
1    P      1            2       True   True
2   D1      1            1       True   True
3   D1      1            2       True   True
4   D2      1            1       True   True
5   D2      1            2       True   True
6    P      2            1       True   True
7    P      2            2       True   True
8   D1      2            1      False  False
9   D1      2            2      False  False
10  D2      2            1      False  False
11  D2      2            2      False  False
12   P      3            1      False  False
13   P      3            2       True   True
14  D1      3            1       True   True
15  D1      3            2       True   True
16  D2      3            1      False  False
17  D2      3            2       True   True


['P', 'D1', 'D2']

In [9]:
from itertools import combinations

a = [1,2,3]

res = [com for sub in range(1,3) for com in combinations(a , sub + 1)]
for comb in res:
    print(comb)
    print(1 in comb)
    print('---')



(1, 2)
True
---
(1, 3)
True
---
(2, 3)
False
---
(1, 2, 3)
True
---


In [25]:
import numpy as np

a = np.empty((10))
a.fill(0)
a

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [32]:
import numpy as np

a = np.array([[1,1,1],[1,0,2]])
np.linalg.norm(a, axis=0, keepdims=False)

array([1.41421356, 1.        , 2.23606798])

In [33]:
test = 1.15
se = np.array([1,1,1])
vals=np.array([1,2,3])
low = vals - se*test
high = vals + se*test
print(low)
print(high)
result = np.array([low,high])
print(result)


[-0.15  0.85  1.85]
[2.15 3.15 4.15]
[[-0.15  0.85  1.85]
 [ 2.15  3.15  4.15]]


In [35]:
a = np.array([[1,0,3],[1,0,0]])
 
np.any(a,axis = 0)

array([ True, False,  True])

In [47]:
'prostate' in 'prostate 1'
'prostate 1' in 'prostate'

False

In [53]:
a = np.array([[1,2,3,4],[1,2,3,4]])
1-a

array([[ 0, -1, -2, -3],
       [ 0, -1, -2, -3]])

In [51]:
a= [(1,2),(3,4),(5,6)]
np.array(a).T

array([[1, 3, 5],
       [2, 4, 6]])

In [81]:
import numpy as np 

a = np.array([[1,1,1],[2,2,2]])

a[[0,1]] = a[[1,0]]

a

array([[2, 2, 2],
       [1, 1, 1]])

In [88]:
import numpy as np 

a = np.array([[1,2,3],[3,1,2],[3,4,9],[30,10,20]])

a[np.array([0,3]),2]    

#np.mean(a,axis=0)

array([ 3, 20])

In [94]:
import pandas as pd

a = pd.DataFrame({'A':np.array([1,2,3]),'b':'Prostate'})
b = pd.DataFrame({'A':np.array([1,2,3]),'b':'DIL'})

frames = [a, b]

result = pd.concat(frames, ignore_index=True)
result

,A,b
0,1,Prostate
1,2,Prostate
2,3,Prostate
3,1,DIL
4,2,DIL
5,3,DIL


In [98]:
import numpy as np 


a = np.array([[1,2,3],[4,5,6]])
a.flatten(order = 'C')
a[:,0]

array([1, 4])

In [100]:
a = {1: False, 2: False, 3: False}
any(a.values())

False

In [122]:
import cudf
df = cudf.DataFrame({'a': [1, 2, None], 'b': [3, None, 5]})
df
df_new = df.fillna(0)
print(df)
print(df_new)

d = df_new['a'].unique()
e = d.to_arrow().to_pylist()
e

      a     b
0     1     3
1     2  <NA>
2  <NA>     5
   a  b
0  1  3
1  2  0
2  0  5


[1, 2, 0]

In [106]:
a = {'a': [1,2,3], 'b': 2, 'c': 'string'}
list(a.values())

[[1, 2, 3], 2, 'string']

In [12]:
import pandas as pd
import numpy as np

a = {'a': [1,2,3], 'b': [4,5,6]}
df = pd.DataFrame(data = a).T
df

a = np.empty((1))
b= np.empty((3))
c = np.append(a,b)
a

array([6.92203645e-310])

In [24]:
import plotly.express as px
import plotly.graph_objects as go

df = px.data.tips()
print(df)

fig = px.box(df, x="day", y="total_bill", color="smoker")
fig2 = px.box(df, x="day", y="tip", color="smoker")
"""
for i in range(0,len(fig2.data)):
    fig.add_trace(fig2.data[i])
fig.update_layout(
        boxmode='group'
    )
"""

fig.update_traces(orientation='h') # horizontal box plots

fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()
"""
fig = go.Figure()
fig.add_trace(go.Box(
        x=df['day'],
        y=df["total_bill"]    ))
fig.show()
"""

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]


'\nfig = go.Figure()\nfig.add_trace(go.Box(\n        x=df[\'day\'],\n        y=df["total_bill"]    ))\nfig.show()\n'

In [26]:
import cupy as cp 
import numpy as np

a = np.array([1,2,3])
b = cp.asnumpy(a)

In [29]:
a = [1,2,3]
import numpy as np

np.std(a)
a.append('b')
a
'b' in a

True

In [31]:
c = {}
c["2"].append(1)
c

KeyError: '2'

In [35]:
a = list(['string'])
a.append('hello')
a

['string', 'hello']